In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 16:24:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
green_revenue_folderpath = 'data/report/revenue/green'
yellow_revenue_folderpath = 'data/report/revenue/yellow'

df_green_revenue = spark.read.parquet(green_revenue_folderpath)
df_yellow_revenue = spark.read.parquet(yellow_revenue_folderpath)

### Joins: green + yellow

In [10]:
df_green_revenue_renamed = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('records_cnt', 'green_records_cnt')
    
df_yellow_revenue_renamed = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('records_cnt', 'yellow_records_cnt')
    
df_all_revenue = df_green_revenue_renamed.join(df_yellow_revenue_renamed, on=['hour', 'zone'], how='outer')

In [15]:
df_all_revenue.show()

+-------------------+----+------------+-----------------+-------------+------------------+
|               hour|zone|green_amount|green_records_cnt|yellow_amount|yellow_records_cnt|
+-------------------+----+------------+-----------------+-------------+------------------+
|2001-01-01 00:00:00| 132|        null|             null|       180.56|                 2|
|2002-12-31 22:00:00| 132|        null|             null|        73.58|                 2|
|2002-12-31 22:00:00| 246|        null|             null|         11.2|                 1|
|2002-12-31 23:00:00| 138|        null|             null|         69.3|                 1|
|2003-01-01 00:00:00| 132|        null|             null|       298.37|                 3|
|2008-12-31 10:00:00| 132|        null|             null|         21.0|                 1|
|2008-12-31 22:00:00| 141|        null|             null|         19.9|                 1|
|2008-12-31 23:00:00|  43|        null|             null|         84.0|                 1|

In [16]:
all_revenue_folderpath = 'data/report/revenue/total'

df_all_revenue.write.parquet(all_revenue_folderpath, mode='overwrite')

### Joins: Fact + Dim

In [17]:
zones_folderpath = 'zones/'

df_zones = spark.read.parquet(zones_folderpath)

In [18]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_all_revenue_zones = df_all_revenue.join(
    df_zones,
    df_all_revenue.zone == df_zones.LocationID
)

In [25]:
df_all_revenue_zones = df_all_revenue_zones.drop('zone', 'LocationID')
df_all_revenue_zones.show()

+-------------------+------------+-----------------+-------------+------------------+---------+------------+
|               hour|green_amount|green_records_cnt|yellow_amount|yellow_records_cnt|  Borough|service_zone|
+-------------------+------------+-----------------+-------------+------------------+---------+------------+
|2001-01-01 00:00:00|        null|             null|       180.56|                 2|   Queens|    Airports|
|2002-12-31 22:00:00|        null|             null|        73.58|                 2|   Queens|    Airports|
|2002-12-31 22:00:00|        null|             null|         11.2|                 1|Manhattan| Yellow Zone|
|2002-12-31 23:00:00|        null|             null|         69.3|                 1|   Queens|    Airports|
|2003-01-01 00:00:00|        null|             null|       298.37|                 3|   Queens|    Airports|
|2008-12-31 10:00:00|        null|             null|         21.0|                 1|   Queens|    Airports|
|2008-12-31 22:00:0

In [26]:
all_revenue_zones_folderpath = 'tmp/revenue-zones'

df_all_revenue_zones.write.parquet(all_revenue_zones_folderpath, mode='overwrite')